In [1]:
from sklearn.datasets import fetch_openml
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
df_ext = pd.read_csv('../data/raw/dating_app_behavior_dataset_extended1.csv')

In [3]:
df = df_ext.copy()

In [4]:
# change target to numerical
df['target_binary'] = df['match_outcome'].isin([
    'Mutual Match',
    'Date Happened',
    'Relationship Formed',
    'Instant Match'
]).astype(int)

In [5]:
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [6]:
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

In [7]:
ordinal_mappings = {
    'income_bracket': [
        'Low', 'Medium', 'High', 'Very High'
    ],
    'education_level': [
        'High School', 'Bachelor', 'Master', 'PhD'
    ],
    'app_usage_time_label': [
        'Very Low', 'Low', 'Medium', 'High', 'Very High'
    ],
    'swipe_right_label': [
        'Never', 'Rarely', 'Sometimes', 'Often', 'Always'
    ]
}

In [10]:
#Encode
from sklearn.preprocessing import OrdinalEncoder

ordinal_cols = list(ordinal_mappings.keys())

encoder = OrdinalEncoder(
    categories=[ordinal_mappings[col] for col in ordinal_cols],
    handle_unknown='use_encoded_value',
    unknown_value=-1
)

df[ordinal_cols] = encoder.fit_transform(df[ordinal_cols])